Group Name: AG xx.

Student Name (Student ID):

1. xxxx xxxxx (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

4. xxxx xxxxx (xxxxxxx)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](./Maze.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the form of a table.
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: The code should be written in a single jupyter notebook file.

In [1]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [2]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 


In [3]:
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
  
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]

def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [4]:
# NOTE: deque need to be imported here!!!
from collections import deque
import heapq
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

In [5]:
MAZE_ROWS = 16
MAZE_COLUMNS = 24
ACTIONS = {'UP', 'LEFT', 'RIGHT', 'DOWN'}
xya = {
    'UP': [1, 0],
    'LEFT': [0, -1],
    'RIGHT': [0, 1],
    'DOWN': [-1, 0],
}
_costs_ = {
    'UP': 1,
    'LEFT': 10,
    'RIGHT': 10,
    'DOWN': 1,
}
_BLACK_ = {
    (14, 9),
    (13, 10),
    (12, 10),
    (11, 10),
    (11, 12),
    (11, 13),
    (10, 9),
    (10, 10),
    (10, 12),
    (10, 13),
    (9, 9),
    (8, 9),
    (7, 9),
    (6, 9),
}
def nxt_xy(state:tuple[int, int], _action_:str)->tuple[int, int]:
    # assert _action_ in ACTIONS
    x, y = state
    ax, ay = xya[_action_]
    return (x+ax, y+ay)

def xy_ok(state:tuple[int, int])->bool:
    nx, ny = state
    ok = (
        (nx >= 0) and (nx < MAZE_ROWS) and 
        (ny >= 0) and (ny < MAZE_COLUMNS) and 
        (state not in _BLACK_)
    )
    return ok

def action_cost(_action_:str)->int:
    # assert _action_ in ACTIONS
    return _costs_[_action_]

class Maze(Problem):
    
    def __init__(self, initial=None, goal=None):
        Problem.__init__(self, initial=initial, goal=goal)

    def h1(self, node):
        """Heuristic 1: Weighted Manhattan distance considering action costs."""
        x1, y1 = node.state
        x2, y2 = self.goal
        vertical_distance = abs(x2 - x1)  # Vertical moves cost 1
        horizontal_distance = abs(y2 - y1)  # Horizontal moves cost 10
        return 1 * vertical_distance + 10 * horizontal_distance

    def h2(self, node):
        """Heuristic 2: Simplified Manhattan distance."""
        x1, y1 = node.state
        x2, y2 = self.goal
        return abs(x2 - x1) + abs(y2 - y1)
    def h3(self, node):
        return node.path_cost
        
    def action_cost(self, s, a, s1): 
        nx, ny = nxt_xy(s, a)
        assert (nx, ny) == s1
        return action_cost(a)
        # raise NotImplementedError                   #Your Code goes here
                  
    def result(self, state, action): 
        nx, ny = nxt_xy(state, action)
        return (nx, ny)
        # raise NotImplementedError                   #Your Code goes here  

    def actions(self, state):
        ret = []
        for a in ACTIONS:
            ns = nxt_xy(state, a)
            if xy_ok(ns):
                ret.append(a)
        return set(ret)
        # raise NotImplementedError                   #Your Code goes here   
    


### Testing the model

In [6]:
#Testing Maze class
#Note that we perform additional tests to validate the modeling

#An instance of Maze problem; initialize the Maze class with appropriate values
m1 = Maze(initial = (8,10), goal = (11,9))

# Testing the methods in Maze class
assert m1.result((4,12),'UP') == (5,12)
assert m1.action_cost((4,12),'UP',(5,12)) == 1
assert m1.actions((4,12)) == {'UP', 'LEFT', 'RIGHT', 'DOWN'}

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

In [7]:
#For breadth-first search
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node_count = 0
    maxfrontier_size = 0
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node,node_count,maxfrontier_size
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        maxfrontier_size = max(maxfrontier_size,len(frontier))
        node = frontier.pop()
        for child in expand(problem, node):
            node_count += 1
            s = child.state
            if problem.is_goal(s):
                return child,node_count,maxfrontier_size
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure,node_count,maxfrontier_size

result_breadth, node_count_breadth, maxfrontier_size_breadth = breadth_first_search(m1) 
print(f'Result:{result_breadth}')
print(f'Number of expanded nodes:{node_count_breadth}')
print(f'Maximum fontier size:{maxfrontier_size_breadth}')
print(f'Path costs: {result_breadth.path_cost}')
print(f'Path states:{[s for s in path_states(result_breadth)]}')


Result:<(11, 9)>
Number of expanded nodes:661
Maximum fontier size:31
Path costs: 39
Path states:[(8, 10), (7, 10), (6, 10), (5, 10), (5, 9), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (11, 9)]


In [8]:
# #For depth-first search with cycle-check
# def is_cycle(node, k):
#     "Does this node form a cycle of length k or less?"
#     def find_cycle(ancestor, k):
#         return (ancestor is not None and k > 0 and
#                 (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
#     return find_cycle(node.parent, k)

# def depth_first_search(problem):
#     "Search deepest nodes in the search tree first."
#     frontier = LIFOQueue([Node(problem.initial)])    
#     result = failure
#     while frontier:
#         node = frontier.pop()
#         if problem.is_goal(node.state):
#             return node
#         elif not is_cycle(node,k=30):
#             for child in expand(problem, node):                
#                 frontier.append(child)
#     return result

# result_dfs = depth_first_search(m1)  
# print(result_dfs.path_cost)
# path_states(result_dfs)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1057dc7a0>>
Traceback (most recent call last):
  File "/Users/christopher/Coding/HKY/IT5008/.conda/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
#Your code for  iterative Deepening search with cycle-check
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])    
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) > limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):                
                frontier.append(child)
    return result

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result

In [ ]:
#Your code for uniform-cost search

In [ ]:
#Your code for A* Search
def astar_search(problem, h=None):
    h = h or problem.h
    frontier = PriorityQueue([Node(problem.initial)], key=lambda node: node.path_cost + h(node))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        explored.add(node.state)
        for child in expand(problem, node):
            if child.state not in explored and child.state not in [n.state for _, n in frontier.items]:
                frontier.add(child)
            elif child.state in [n.state for _, n in frontier.items]:
                index = next((i for i, (_, n) in enumerate(frontier.items) if n.state == child.state), None)
                if index is not None:
                    incumbent = frontier.items[index][1]
                    if child.path_cost < incumbent.path_cost:
                        frontier.items[index] = (child.path_cost + h(child), child)
                        heapq.heapify(frontier.items)
    return failure

# use A* search and heuristic 1 to solve
m = Maze(initial = (8, 10), goal = (11, 9))
result_astar = astar_search(m,m.h3)
print(result_astar, result_astar.path_cost)
for s in path_states(result_astar):
    print(s)

<(11, 9)> 39
(8, 10)
(7, 10)
(6, 10)
(5, 10)
(5, 9)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
(11, 8)
(11, 9)


In [ ]:
#Your code for greedy-best first search
def greedy_bfs(problem, h=None):
    """以 h(n) 最小的节点进行搜索。"""
    h = h or problem.h
    frontier = PriorityQueue([Node(problem.initial)], key=h)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        explored.add(node.state)
        for child in expand(problem, node):
            if child.state not in explored and child.state not in [n.state for _, n in frontier.items]:
                frontier.add(child)
    return failure

# use greedy best_first search and heuristic 2 to solve
Maze.h = Maze.h2
m = Maze(initial = (8, 10), goal = (11, 9))
result_greedy = greedy_bfs(m)
print(result_greedy, result_greedy.path_cost)
for s in path_states(result_greedy):
    print(s)

<(11, 9)> 61
(8, 10)
(9, 10)
(9, 11)
(10, 11)
(11, 11)
(12, 11)
(13, 11)
(14, 11)
(14, 10)
(15, 10)
(15, 9)
(15, 8)
(14, 8)
(13, 8)
(12, 8)
(11, 8)
(11, 9)


2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the following table.


 Algorithm             |Number of expanded nodes  | Maximum Frontier Size  |  Path Cost                  |
|:---------------------|:-------------------------|:-----------------------|:----------
| Breadth-First Search |                          |                        |
| DFS with cycle check |                          |                        |
| IDS with cycle check |                          |                        |
| UCS                  |                          |                        |
| A* Search            |                          |                        |
| GBFS                 |                          |                        |

3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.

